In [1]:
import pandas as pd
import requests
from pathlib import Path
from datetime import datetime
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

BASE_URL = "https://chilamboli.devmorphix.com"
CATEGORY_TO_LETTER = {"Sub Junior": "B", "Junior": "J", "Senior": "S", "Combined": "C", "Special": "P"}

output_dir = Path("output/chest_numbers").resolve()
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Track event letters within each category (A, B, C, ...)
_event_letters = {}  # Maps (category, event_name) -> letter
_category_event_counters = {}  # Tracks how many events per category
_chest_counters = {}  # Maps (category, event_letter) -> counter (starts from 101)

def get_category_letter(age_category):
    """Get the category letter (S, J, B, C, P)"""
    return CATEGORY_TO_LETTER.get(age_category.strip())

def get_event_letter(category_letter, event_name):
    """Get or assign a letter (A, B, C, ...) for an event within a category"""
    key = (category_letter, event_name.strip())
    
    if key not in _event_letters:
        # Assign next available letter for this category
        category_count = _category_event_counters.get(category_letter, 0)
        event_letter = chr(ord('A') + category_count)  # A, B, C, ...
        _event_letters[key] = event_letter
        _category_event_counters[category_letter] = category_count + 1
    
    return _event_letters[key]

def parse_chest_number(chest_num):
    """Parse chest number like 'SA101' into (category, event_letter, number)
    Returns (cat, event_letter, number) or None if invalid"""
    if not chest_num or len(chest_num) < 4:
        return None
    try:
        cat = chest_num[0]  # First char: S, J, B, C, P
        event_letter = chest_num[1]  # Second char: A, B, C, ...
        number = int(chest_num[2:])  # Rest: 101, 102, etc.
        return (cat, event_letter, number)
    except (ValueError, IndexError):
        return None

def initialize_counters_from_existing(registrations):
    """Initialize counters based on existing chest numbers"""
    # First pass: map events to their letters and track max numbers
    for reg in registrations:
        chest_num = reg.get("chestNumber")
        if not chest_num:
            continue
        event = reg.get("event", {})
        event_name = event.get("name", "").strip()
        age_category = event.get("ageCategory", "").strip()
        parsed = parse_chest_number(chest_num)
        if parsed:
            cat, event_letter, number = parsed
            # Map event to its letter
            key = (cat, event_name)
            if key not in _event_letters:
                _event_letters[key] = event_letter
            
            # Track max number for each (cat, event_letter) combination
            counter_key = (cat, event_letter)
            if counter_key not in _chest_counters:
                _chest_counters[counter_key] = number
            else:
                _chest_counters[counter_key] = max(_chest_counters[counter_key], number)
    
    # Second pass: calculate category event counters based on unique letters used
    for cat in set(c for c, _ in _event_letters.keys()):
        unique_letters = set()
        for (c, en), el in _event_letters.items():
            if c == cat:
                unique_letters.add(el)
        # Find the highest letter index used (A=0, B=1, C=2, etc.)
        max_letter_idx = -1
        for letter in unique_letters:
            idx = ord(letter) - ord('A')
            max_letter_idx = max(max_letter_idx, idx)
        _category_event_counters[cat] = max_letter_idx + 1  # Next available index
    
    # Third pass: initialize counters for events without existing chest numbers
    # This ensures we start from 101 for new event/category combinations
    for reg in registrations:
        event = reg.get("event", {})
        event_name = event.get("name", "").strip()
        age_category = event.get("ageCategory", "").strip()
        cat = get_category_letter(age_category)
        if not cat:
            continue
        key = (cat, event_name)
        if key not in _event_letters:
            # Assign next available letter
            category_count = _category_event_counters.get(cat, 0)
            event_letter = chr(ord('A') + category_count)
            _event_letters[key] = event_letter
            _category_event_counters[cat] = category_count + 1
            # Initialize counter at 100 (will be incremented to 101 on first use)
            counter_key = (cat, event_letter)
            if counter_key not in _chest_counters:
                _chest_counters[counter_key] = 100

def next_chest_number(event_name, age_category):
    """Generate next chest number: {category}{event_letter}{number}
    Format: SA101, SA102, ..., SB101, SB102, etc.
    Numbers start from 101"""
    cat = get_category_letter(age_category)
    if not cat:
        return None
    
    event_letter = get_event_letter(cat, event_name)
    counter_key = (cat, event_letter)
    
    # Initialize counter at 101 if first time
    if counter_key not in _chest_counters:
        _chest_counters[counter_key] = 101
    else:
        _chest_counters[counter_key] += 1
    
    number = _chest_counters[counter_key]
    chest_num = f"{cat}{event_letter}{number}"
    return chest_num, chest_num

def reset_counters():
    """Reset all counters and mappings"""
    _event_letters.clear()
    _category_event_counters.clear()
    _chest_counters.clear()

def _fetch_paginated(path, limit=100, **params):
    """Fetch all data from paginated API endpoint"""
    page, all_data = 1, []
    while True:
        r = requests.get(f"{BASE_URL}{path}", params={**params, "page": page, "limit": limit}, timeout=30)
        r.raise_for_status()
        j = r.json()
        # Handle both response formats: with/without "success" field
        if "success" in j and not j.get("success"):
            raise RuntimeError(j.get("message", "API error"))
        data = j.get("data", [])
        all_data.extend(data)
        meta = j.get("metadata", {})
        total = meta.get("total", 0)
        if len(all_data) >= total or not data:
            break
        page += 1
    return all_data

print("Helpers defined: get_category_letter, get_event_letter, next_chest_number, reset_counters, _fetch_paginated")

In [ ]:
ts = datetime.now().strftime("%Y%m%d_%H%M%S")

all_regs = _fetch_paginated("/api/registrations")
school_map = {s["id"]: s.get("name", "Unknown") for s in _fetch_paginated("/api/schools")}

reset_counters()
initialize_counters_from_existing(all_regs)

new_chest = {}
for reg in all_regs:
    if reg.get("chestNumber"):
        continue
    event = reg.get("event", {})
    result = next_chest_number(event.get("name", ""), event.get("ageCategory", ""))
    if result:
        chest = result[0]
        new_chest[reg["id"]] = chest

# SQL updates (D1 compatible)
if new_chest:
    sql = ["-- Chest number updates", f"-- {datetime.now():%Y-%m-%d %H:%M:%S}", ""]
    for rid, c in sorted(new_chest.items()):
        sql.append(f"UPDATE registrations SET chest_number = '{c.replace(chr(39), chr(39)+chr(39))}' WHERE id = '{rid}';")
    (output_dir / f"update_chest_numbers_{ts}.sql").write_text("\n".join(sql), encoding="utf-8")
    print(f"✓ SQL: {len(new_chest)} updates")
else:
    print("No new chest numbers to generate.")

# Event letters
print("\nEvent letters:", ", ".join(f"{c}{l}: {n}" for (c, n), l in sorted(_event_letters.items())))

# Excel: new only
if new_chest:
    rows = []
    for reg in all_regs:
        if reg.get("id") not in new_chest:
            continue
        e = reg.get("event", {})
        rows.append({
            "Registration ID": reg["id"], "Chest Number": new_chest[reg["id"]],
            "Event Name": e.get("name", ""), "Age Category": e.get("ageCategory", ""),
            "Team Name": reg.get("teamName", ""), "School Name": school_map.get(reg.get("schoolId"), "Unknown"),
        })
    df_new = pd.DataFrame(rows).sort_values(["Age Category", "Event Name", "Chest Number"])
    df_new.to_excel(output_dir / f"chest_numbers_new_{ts}.xlsx", sheet_name="Chest Numbers", index=False)
    print(f"✓ Excel (new): {len(df_new)} rows")

# Excel: all
rows = []
for reg in all_regs:
    e = reg.get("event", {})
    chest = reg.get("chestNumber") or new_chest.get(reg["id"], "")
    if not chest and not get_category_letter(e.get("ageCategory", "")):
        continue
    rows.append({
        "Chest Number": chest, "Event Name": e.get("name", ""), "Age Category": e.get("ageCategory", ""),
        "Team Name": reg.get("teamName", ""), "School Name": school_map.get(reg.get("schoolId"), "Unknown"),
    })
df_all = pd.DataFrame(rows).sort_values(["Age Category", "Event Name", "Chest Number"])
df_all.to_excel(output_dir / f"chest_numbers_all_{ts}.xlsx", sheet_name="Chest Numbers", index=False)
print(f"✓ Excel (all): {len(df_all)} rows")